In [1]:
// declare our variables up here
let prompt, files, issue, i;


undefined

In [2]:
issue = `@file1:/path/to/file1
@file2:/path/to/file2

The amount of CPU in @file1 needs to be increased to 512M
The PVC storage amount in @file2 should be decreased to 10Gi
`;

'@file1:/path/to/file1\n' +
  '@file2:/path/to/file2\n' +
  '\n' +
  'The amount of CPU in @file1 needs to be increased to 512M\n' +
  'The PVC storage amount in @file2 should be decreased to 10Gi\n'

In [3]:
files = new Map([
  ['file1', {
    path: 'grafana/base/datasource.yaml',
    content: 'apiVersion: integreatly.org/v1alpha1\n' +
      'kind: GrafanaDataSource\n' +
      'metadata:\n' +
      '  name: datasource\n' +
      'spec:\n' +
      '  name: prometheus-grafanadatasource.yaml\n' +
      '  datasources:\n' +
      '    - name: Prometheus\n' +
      '    - access: proxy\n' +
      '      editable: true\n' +
      '      isDefault: true\n' +
      '      jsonData:\n' +
      "        httpHeaderName1: 'Authorization'\n" +
      '        timeInterval: 5s\n' +
      '        tlsSkipVerify: true\n' +
      '      name: Prometheus\n' +
      '      secureJsonData:\n' +
      "        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'\n" +
      '      type: prometheus\n' +
      "      url: 'https://thanos-querier.openshift-monitoring.svc.cluster.local:9091'\n",
    updatedContent: ''
  }],
  ['file2', {
    path: 'grafana/base/grafana-route.yaml',
    content: 'kind: Route\n' +
      'apiVersion: route.openshift.io/v1\n' +
      'metadata:\n' +
      '  name: grafana\n' +
      '  annotations:\n' +
      '    kubernetes.io/tls-acme: "true"\n' +
      'spec:\n' +
      '  host: grafana.operate-first.cloud\n' +
      '  to:\n' +
      '    kind: Service\n' +
      '    name: grafana-service\n' +
      '  port:\n' +
      '    targetPort: 3000\n',
    updatedContent: ''
  }]
]);

Map(2) {
  'file1' => {
    path: 'grafana/base/datasource.yaml',
    content: 'apiVersion: integreatly.org/v1alpha1\n' +
      'kind: GrafanaDataSource\n' +
      'metadata:\n' +
      '  name: datasource\n' +
      'spec:\n' +
      '  name: prometheus-grafanadatasource.yaml\n' +
      '  datasources:\n' +
      '    - name: Prometheus\n' +
      '    - access: proxy\n' +
      '      editable: true\n' +
      '      isDefault: true\n' +
      '      jsonData:\n' +
      "        httpHeaderName1: 'Authorization'\n" +
      '        timeInterval: 5s\n' +
      '        tlsSkipVerify: true\n' +
      '      name: Prometheus\n' +
      '      secureJsonData:\n' +
      "        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'\n" +
      '      type: prometheus\n' +
      "      url: 'https://thanos-querier.openshift-monitoring.svc.cluster.local:9091'\n",
    updatedContent: ''
  },
  'file2' => {
    path: 'grafana/base/grafana-route.yaml',
    content: 'kind: Route\n' +
      'apiVersion: ro

## Building the prompt

In [4]:
prompt = `# Listed below are:
# 1. Explanation of how two files need to be changed
# 2. The original files, separated by a '---' string
# 3. An updated version of the files with the described changes, with each file separated by a '---' string
# 4. A '####' string, indicating the end of this document


## Description of issues:
${issue}\n

## Original files:
`;

for (const [fileName, file] of files) {
	prompt += `# @${fileName}\n${file.content}\n`;
	// only place the delimiting string if in-between files
	if (files.size > 1 && i < files.size - 1) {
		prompt += '---\n';
	}
	i++;
}

prompt += `
## Updated files:
`;


'# Listed below are:\n' +
  '# 1. Explanation of how two files need to be changed\n' +
  "# 2. The original files, separated by a '---' string\n" +
  "# 3. An updated version of the files with the described changes, with each file separated by a '---' string\n" +
  "# 4. A '####' string, indicating the end of this document\n" +
  '\n' +
  '\n' +
  '## Description of issues:\n' +
  '@file1:/path/to/file1\n' +
  '@file2:/path/to/file2\n' +
  '\n' +
  'The amount of CPU in @file1 needs to be increased to 512M\n' +
  'The PVC storage amount in @file2 should be decreased to 10Gi\n' +
  '\n' +
  '\n' +
  '\n' +
  '## Original files:\n' +
  '# @file1\n' +
  'apiVersion: integreatly.org/v1alpha1\n' +
  'kind: GrafanaDataSource\n' +
  'metadata:\n' +
  '  name: datasource\n' +
  'spec:\n' +
  '  name: prometheus-grafanadatasource.yaml\n' +
  '  datasources:\n' +
  '    - name: Prometheus\n' +
  '    - access: proxy\n' +
  '      editable: true\n' +
  '      isDefault: true\n' +
  '      jsonDat